In [1]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime #langchain_community#tiktoken - tokenization of data

In [2]:
!pip install langchain-community


In [3]:
WEAVIATE_API_KEY = "LRt5io5sKq1IaUCamCQoFjM0QUrxBymEvRBO"
WEAVIATE_CLUSTER = "https://my-app-zxhwdmsd.weaviate.network"

In [4]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY= WEAVIATE_API_KEY

client= weaviate.Client(
    url = WEAVIATE_URL,
    auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)

)

In [5]:
# to fix unicode error in any url or any string
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
!pip install sentence-transformers

In [7]:
#embedding model available in hugging face
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(
    model_name = embedding_model_name
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from langchain.document_loaders import PyPDFLoader #to import pdf
loader = PyPDFLoader("/content/pdf1.pdf",extract_images= True)
pages = loader.load()


In [10]:
pages

[Document(page_content='Active Retrieval Augmented Generation\nZhengbao Jiang1∗Frank F. Xu1∗Luyu Gao1∗Zhiqing Sun1∗Qian Liu2\nJane Dwivedi-Yu3Yiming Yang1Jamie Callan1Graham Neubig1\n1Language Technologies Institute, Carnegie Mellon University\n2Sea AI Lab3FAIR, Meta\n{zhengbaj,fangzhex,luyug,zhiqings,gneubig}@cs.cmu.edu\nAbstract\nDespite the remarkable ability of large lan-\nguage models (LMs) to comprehend and gen-\nerate language, they have a tendency to hal-\nlucinate and create factually inaccurate out-\nput. Augmenting LMs by retrieving informa-\ntion from external knowledge resources is one\npromising solution. Most existing retrieval aug-\nmented LMs employ a retrieve-and-generate\nsetup that only retrieves information once based\non the input. This is limiting, however, in\nmore general scenarios involving generation\nof long texts, where continually gathering in-\nformation throughout generation is essential. In\nthis work, we provide a generalized view of ac-\ntive retrieva

In [11]:
#split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter #this is used to make chunks of the data.
text_splitter= RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap=20)

#passing the documents to text_splitter
docs = text_splitter.split_documents(pages) #splitting the document into various chunks.#splitting the doc in small sentences
#chunks are essentially the piece of sentences from the data.

In [12]:
docs

[Document(page_content='Active Retrieval Augmented Generation\nZhengbao Jiang1∗Frank F. Xu1∗Luyu Gao1∗Zhiqing Sun1∗Qian Liu2\nJane Dwivedi-Yu3Yiming Yang1Jamie Callan1Graham Neubig1\n1Language Technologies Institute, Carnegie Mellon University\n2Sea AI Lab3FAIR, Meta\n{zhengbaj,fangzhex,luyug,zhiqings,gneubig}@cs.cmu.edu\nAbstract\nDespite the remarkable ability of large lan-\nguage models (LMs) to comprehend and gen-\nerate language, they have a tendency to hal-\nlucinate and create factually inaccurate out-\nput. Augmenting LMs by retrieving informa-\ntion from external knowledge resources is one\npromising solution. Most existing retrieval aug-\nmented LMs employ a retrieve-and-generate\nsetup that only retrieves information once based\non the input. This is limiting, however, in\nmore general scenarios involving generation\nof long texts, where continually gathering in-\nformation throughout generation is essential. In\nthis work, we provide a generalized view of ac-\ntive retrieva

In [13]:
vector_db = Weaviate.from_documents(docs, embeddings, client=client, by_text=False)

## RETRIEVAL


In [14]:
vector_db.similarity_search("what is attention?",k=3) #k=3 means how many simialr sentences that we want. #gives ranked results
#k=3 gives the number of similar sentences based on the rank.

[Document(page_content='Zhiruo Wang, Jamie Callan, and Graham Neubig.\n2022. Retrieval as attention: End-to-end learning\nof retrieval and reading within a single transformer.\nCoRR , abs/2212.02027.\nZhengbao Jiang, Frank F. Xu, Jun Araki, and Graham\nNeubig. 2020. How can we know what language\nmodels know. Trans. Assoc. Comput. Linguistics ,\n8:423–438.\nMandar Joshi, Eunsol Choi, Daniel S. Weld, and Luke\nZettlemoyer. 2017. Triviaqa: A large scale distantly\nsupervised challenge dataset for reading comprehen-\nsion. In Proceedings of the 55th Annual Meeting of\nthe Association for Computational Linguistics, ACL\n2017, Vancouver, Canada, July 30 - August 4, Volume\n1: Long Papers , pages 1601–1611. Association for\nComputational Linguistics.\nSaurav Kadavath, Tom Conerly, Amanda Askell, Tom\nHenighan, Dawn Drain, Ethan Perez, Nicholas\nSchiefer, Zac Hatfield-Dodds, Nova DasSarma, Eli\nTran-Johnson, Scott Johnston, Sheer El Showk, Andy\nJones, Nelson Elhage, Tristan Hume, Anna Chen,'

In [15]:
print(vector_db.similarity_search("what is rag?",k=4))

[Document(page_content='gon) including the following aspects: academics,\nhistory.”, the output we aim to generate is “# Aca-\ndemics. In 2008, 91% of the school’s seniors re-\nceived their high school diploma... # History. The\nclass of 2008 was the 100th class in the school’s\nhistory.” where # is used to indicate aspects. We\nmanually annotate 4 exemplars (Prompt D.10), and\nuse the Bing search engine to retrieve 5 documents\nfrom the open web. To avoid leaking, we exclude\nseveral Wikipedia-related domains listed in Table 8\nfrom Bing’s search results.\nC Hyperparameters\nHyperparameters of FLARE on different datasets\nare listed in Table 9.\nD Prompts and Few-shot exemplars\nThe prompt used to linearize multiple documents\nis shown in Prompt D.1. The prompt used in self-\nask (Press et al., 2022) is shown in Prompt D.2.\nPrompts and exemplars of different tasks/datasets\nare shown in Prompt D.3, D.4, D.5, D.6, D.8, and\nD.10, respectively.\nPrompt D.1: document formatting\nSearch 

In [16]:
print(vector_db.similarity_search("what is genai?",k=2)[0].page_content)

to retrieve extra information (Press et al., 2022; Yao
et al., 2022; Khot et al., 2022; Khattab et al., 2022).
We ask the following question: can we create a
simple and generic retrieval augmented LM that ac-
tively decides when and what to retrieve throughout
the generation process, and are applicable to a va-
riety of long-form generation tasks? We provide a
generalized view of active retrieval augmented gen-
eration. Our hypothesis regarding when to retrieve
is that LMs should retrieve information only whenthey lack the required knowledge to avoid unneces-
sary or inappropriate retrieval that occurs in passive
retrieval augmented LMs (Khandelwal et al., 2020;
Borgeaud et al., 2022; Ram et al., 2023; Trivedi
et al., 2022). Given the observation that large LMs
tend to be well-calibrated and low probability/con-
fidence often indicates a lack of knowledge (Ka-
davath et al., 2022), we adopt an active retrieval
strategy that only retrieves when LMs generate low-


## Generating the response from LLM


In [17]:

from langchain.prompts import ChatPromptTemplate

template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:


"""

In [18]:
#context meaning is the retriveal the vector database

In [19]:
prompt =ChatPromptTemplate.from_template(template)

In [20]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context} \nAnswer:\n\n\n"))])

In [ ]:
#Downloading the mistral-7b api key from hugging face.

In [21]:
from langchain import HuggingFaceHub

In [25]:
from google.colab import userdata
huggingfacehub_api_token=userdata.get('hugging-face')

In [ ]:
model = HuggingFaceHub(  #llm model
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)


In [27]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [29]:
retriever = vector_db.as_retriever()

In [28]:
output_parser = StrOutputParser() #in the response we'll get output in proper format.

In [ ]:
rag_chain=(

           {"context":retriever, "question":RunnablePassthrough()}
           | prompt
           | model
           | output_parser

)

#combining prompt, model and output parser using the expression language of the langchain by creating the chain,

In [ ]:
rag_chain.invoke("what is rag application?")